In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

In [2]:
## parameters cell
itp_id = 135

In [3]:
# Parameters
district = "04 - Oakland"
itp_id = 246


In [4]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [5]:
# airtable_organizations

In [6]:
analysis_date = dt.date(2022, 3, 30)
if itp_id == 273:
    analysis_date = dt.date(2022, 2, 8) # #override for SacRT 

In [7]:
month_day = analysis_date.strftime('%m_%d')

In [8]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_{month_day}.parquet')

In [9]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_{month_day}.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [10]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [11]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [12]:
%%capture_parameters
agency_name = rt_day.calitp_agency_name
district = airtable_organizations.caltrans_district.iloc[0]
agency_name, district

# Caltrain

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for Caltrain, with a map each for the morning peak, midday, and afternoon peak periods.
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel.
* Hover over, or click, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

In [13]:
## avoid papermill errors when map fails to render
m = None

## Morning Peak

In [14]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [15]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [16]:
m

In [17]:
rt_day.describe_delayed_routes()

AM Peak most delayed routes: 
 * L6, LTD 6, Northbound: 16.0 minutes late on average
 * L6, LTD 6, Northbound: 8.0 minutes late on average
 * L6, LTD 6, Northbound: 5.0 minutes late on average
 * L6, LTD 6, Southbound: 1.0 minutes late on average
 * L6, LTD 6, Southbound: 0.0 minutes late on average

## Midday

In [18]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00')

In [19]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [20]:
m

In [21]:
rt_day.describe_delayed_routes()

Midday most delayed routes: 
 * L5, LTD 5, Southbound: 8.0 minutes late on average
 * L1, Local, Southbound: 5.0 minutes late on average
 * L1, Local, Northbound: 4.0 minutes late on average
 * L5, LTD 5, Northbound: 2.0 minutes late on average

## Afternoon Peak

In [22]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [23]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds', no_title=True)

In [24]:
m

In [25]:
rt_day.describe_delayed_routes()

PM Peak most delayed routes: 
 * L6, LTD 6, Southbound: 6.0 minutes late on average
 * L1, Local, Southbound: 6.0 minutes late on average
 * L6, LTD 6, Northbound: 4.0 minutes late on average
 * L6, LTD 6, Northbound: 3.0 minutes late on average
 * L6, LTD 6, Southbound: 2.0 minutes late on average

In [26]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance)
         >> arrange(-_.speed_variance)
         >> head(20)
        )
except Exception as e:
    pass
    # print(e)
## check that route has all-day span...

In [27]:
def map_from_metrics(rt_day, metrics_df):
    try:
        route_name = metrics_df.iloc[1,0]
        direction_id = metrics_df.iloc[1,1]
        # print(route_name, direction_id)
    except Exception as e:
        # print(e)
        return
    # route_name = '281'
    # direction_id = '1'
    rt_day.set_filter(start_time='06:00', end_time='09:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        am_map = rt_day.segment_speed_map(how='low_speeds')
        display(am_map)
    except Exception as e:
        # print(e)
        pass
    
    rt_day.set_filter(start_time='10:00', end_time='14:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        mid_map = rt_day.segment_speed_map(how='low_speeds')
        display(mid_map)
    except Exception as e:
        # print(e)
        pass

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name], direction_id = direction_id)
    try:
        pm_map = rt_day.segment_speed_map(how='low_speeds')
        display(pm_map)
    except Exception as e:
        # print(e)
        pass

    return

## Single-Route Detail Example

In [28]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    # print(e)
    pass